In [1]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

def extract_record(item):
    """extract and return result from a single result"""
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.in' + atag.get('href')
    
    try:
        #price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        #rating
        rating = item.i.text
    except AttributeError:
        return
    
    result = (description, price, rating, url)
    
    return result

#moving to next page
def next_page_url():
    anc_tag = soup.find('li',{"class":"a-last"})
    next_url = anc_tag.a
    nurl = 'https://www.amazon.in' + next_url.get('href')
    return nurl

In [2]:
driver = webdriver.Chrome()
records = []
    
for page in range(1,8):
    if page==1:
        url = 'https://www.amazon.in/s?k=mobile+phone+under+20000&crid=32XZ005X0CN9R&sprefix=mobile+pho%2Caps%2C313&ref=nb_sb_noss_2'
    else:
        url = next_page_url()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    results = soup.find_all('div',{"data-component-type":"s-search-result"})
    
    for item in results:
        record = extract_record(item)
        if record:
            records.append(record)
            
driver.close()

In [3]:
records

[('OPPO A52 (Stream White, 6GB RAM, 128GB Storage) with No Cost EMI/Additional Exchange Offers',
  '₹13,990',
  '3.9 out of 5 stars',
  'https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A02594861DR9MC43AOVLL&url=%2FStream-Storage-Additional-Exchange-Offers%2Fdp%2FB086KCD5YS%2Fref%3Dsr_1_1_sspa%3Fcrid%3D32XZ005X0CN9R%26dchild%3D1%26keywords%3Dmobile%2Bphone%2Bunder%2B20000%26qid%3D1604035397%26sprefix%3Dmobile%2Bpho%252Caps%252C313%26sr%3D8-1-spons%26psc%3D1&qualifier=1604035397&id=2393798223127072&widgetName=sp_atf'),
 ('Oppo A31 (Mystery Black, 6GB RAM, 128GB Storage) with No Cost EMI/Additional Exchange Offers',
  '₹11,990',
  '4.0 out of 5 stars',
  'https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0679328GVLK4Q8UB1NF&url=%2FOppo-Mystery-Storage-Additional-Exchange%2Fdp%2FB08444S68L%2Fref%3Dsr_1_2_sspa%3Fcrid%3D32XZ005X0CN9R%26dchild%3D1%26keywords%3Dmobile%2Bphone%2Bunder%2B20000%26qid%

In [4]:
#save data in csv file
with open('smartphone_results.csv','w', newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Description', 'Price', 'Rating', 'URL'])
    writer.writerows(records)